In [1]:
import os
from pathlib import Path
import numpy as np
import osmnx as ox
import geopandas as gpd
from shapely.geometry import MultiPolygon
from lib.city_blocks import load_street_graph, remove_deadends, city_blocks
%matplotlib inline  

## Intro
This is a notebook that shows how to find city blocks from a street graph. The main code used in this notebook is found in `city_blocks.py`

## Settings

In [2]:
### Area settings
radius = 2500
coords_nakano = (35.6059402,139.6664317)
network_type = 'drive'

### Speed-up settings
remove_road_curves = False
remove_dead_ends = True
cityblock_use_cached = False

data_prefix = '{}-{}-{}'.format(coords_nakano, radius, network_type)

## Load street graph
Use cached data if available, otherwise download first and then save to disk.  
NOTE: This will take a while if a large radius is used! 

In [3]:
street_graph = load_street_graph(coords=coords_nakano, 
                                 radius=radius, 
                                 network_type=network_type,
                                 filename=data_prefix+'.graphml')

downloading street graph
(35.6059402, 139.6664317) 2500 drive


TypeError: unhashable type: 'dict'

In [ ]:
ox.plot_graph(street_graph);

## Simplify graph using `osmnx` simplification
Removes nodes to join edges that make up road curves. Note that doing this will affect the resulting city blocks. This is can be ammended by a different implementation. 

In [ ]:
if remove_road_curves:
    street_graph = ox.simplify_graph(street_graph)

In [ ]:
ox.plot_graph(street_graph);

## Simplify graph by removing dead-end roads
Removing the dead-end roads does not affect the shape of the city blocks. 

In [ ]:
if remove_dead_ends:
    street_graph = remove_deadends(street_graph)

We need to check that there's any street network left after applying our simplifications:

In [ ]:
assert len(street_graph) > 0

In [ ]:
ox.plot_graph(street_graph);

Plotting the simplified graph in folium is handy for inspecting local areas. Slow for large areas. 

In [ ]:
#ox.plot_graph_folium(street_graph)

## Calculate city blocks

In [ ]:
blocks = city_blocks(street_graph)

In [ ]:
blocks.plot(figsize=(15,15));

In [ ]:
print('number of areas: {}'.format(len(blocks)))

## Dealing with invalid polygons

When calculating city blocks for larger areas, it will often happen that some of the polygons representing the city blocks are invalid. This means that the polygons are complex and have self-intersections. We can deal with this by applying some shapely magic.  

First, we find the invalid polygons and plot them.

In [ ]:
invalid = blocks[blocks.is_valid == False]
if not invalid.empty: 
    invalid.plot(figsize=(10,10))

Fix polygons and create second dataframe

In [ ]:
fixed = [ ]
for _,pol in invalid.geometry.iteritems():
    pol = pol.buffer(0)
    if isinstance(pol, MultiPolygon):
        for p in pol.geoms:
            fixed.append(p)
    else:
        fixed.append(pol)
fixed = gpd.GeoDataFrame({'geometry':fixed})
       

In [ ]:
blocks = blocks[blocks.is_valid == True]
blocks = blocks.append(fixed)
blocks.reset_index(inplace=True, drop=True)

And voila!

In [ ]:
blocks.plot(figsize=(15,15));

## Save areas as GeoJson

In [ ]:
geofile = Path(tmp, data_prefix).with_suffix('.geojson')

with geofile.open('w') as af: 
    af.write(blocks.to_json())

Geojson file size in MiB:

In [ ]:
os.path.getsize(geofile.as_posix())/2**20